# Example of re-expansion of the H₂CO PES using identical internal coordinate system

Here, we test expansion of the PES for formaldehyde molecule using different types of internal coordinates

In [1]:
import sys
sys.path.insert(1, '../')
import os

In [2]:
import jax
import jax.numpy as jnp
import numpy as np

from vibrojet.basis_utils import generate_prod_ind
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

### Expand PES using the following coordinates:
- $y_1=1-\exp[-(r_{CO}-r_{CO}^{eq}(\tau))]$
- $y_2=1-\exp[-(r_{CH_1}-r_{CH_1}^{eq}(\tau))]$
- $y_3=1-\exp[-(r_{CH_2}-r_{CH_2}^{eq}(\tau))]$
- $y_4=\alpha_{\angle OCH_1}-\alpha_{\angle OCH_1}^{eq}(\tau)$
- $y_5=\alpha_{\angle OCH_2}-\alpha_{\angle OCH_2}^{eq}(\tau)$
- $y_6 = \cos(\tau)+1$

The equilibrium values of internal coordinates $r_{CO}^{eq}(\tau)$, ..., $\alpha_{\angle OCH_2}^{eq}(\tau)$ as functions of the dihedral *book* angle $\tau$ or $\cos(\tau)$ are calculated by function `h2co_AYTY.mep(tau)` or `h2co_AYTY.mep(cos_tau, if_costau=True)`, respectively.

In [3]:
def internal_to_y(q):
    r0, r1, r2, a1, a2, cos_tau = q
    r0eq, r1eq, r2eq, a1eq, a2eq, _ = h2co_AYTY.mep(cos_tau, if_costau=True)
    y1 = 1 - jnp.exp(-(r0 - r0eq))
    y2 = 1 - jnp.exp(-(r1 - r1eq))
    y3 = 1 - jnp.exp(-(r2 - r2eq))
    y4 = a1 - a1eq
    y5 = a2 - a2eq
    y6 = 1 + cos_tau
    return jnp.array([y1, y2, y3, y4, y5, y6])


def y_to_internal(y):
    y1, y2, y3, y4, y5, y6 = y
    cos_tau = y6 - 1
    r0eq, r1eq, r2eq, a1eq, a2eq, _ = h2co_AYTY.mep(cos_tau, if_costau=True)
    r0 = -jnp.log(1 - y1) + r0eq
    r1 = -jnp.log(1 - y2) + r1eq
    r2 = -jnp.log(1 - y3) + r2eq
    a1 = y4 + a1eq
    a2 = y5 + a2eq
    return jnp.array([r0, r1, r2, a1, a2, cos_tau])

Generate expansion for PES

In [4]:
ncoo = 6

max_pow = 6
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("max expansion power:", max_pow)
print("number of expansion terms:", len(deriv_ind))

tau0 = np.pi  # equilibrium value of the `book` dihedral angle
cos_tau0 = np.cos(tau0)

q0 = h2co_AYTY.mep(cos_tau0, if_costau=True)
y0 = internal_to_y(q0)

print("equilibrium values of r-alpha-costau coordinates:", q0)
print("equilibrium values of morse-dalpha-dcostau coordinates:", y0)

#h2co_AYTY.poten(y_to_internal(y0), if_costau=True)

poten_coefs = deriv_list(
    lambda x: h2co_AYTY.poten(y_to_internal(x), if_costau=True),
    deriv_ind,
    y0,
    if_taylor=True,
)

max expansion power: 6
number of expansion terms: 924
equilibrium values of r-alpha-costau coordinates: [ 1.2   1.1   1.1   2.13  2.13 -1.  ]
equilibrium values of morse-dalpha-dcostau coordinates: [0. 0. 0. 0. 0. 0.]
Time for d= 0 : 7.32 s
Time for d= 1 : 7.32 s
Time for d= 2 : 14.26 s
Time for d= 3 : 24.09 s
Time for d= 4 : 37.73 s
Time for d= 5 : 53.36 s
Time for d= 6 : 78.39 s


Print computed coefficients and compare with the original PES expansion coefficients, with which they must agree

In [11]:
thresh = 1e-6
max_diff = 0
for i in range(len(deriv_ind)):
    #if np.abs(poten_coefs[i] > thresh):
    if np.sum(np.prod(h2co_AYTY.ind == deriv_ind[i],axis=1)) == 1:
        ind = np.where((h2co_AYTY.ind == deriv_ind[i]).all(axis=1))[0][0]
        coef = h2co_AYTY.coefs[ind]
        diff = np.abs(poten_coefs[i] - coef)
        max_diff = max(max_diff, diff)
        print(
            deriv_ind[i],
            "%20.6f" % poten_coefs[i],
            "%20.6f" % coef,
            "%20.6f" % diff,
        )
    #else:
        #print(poten_coefs[i])
            
print("max difference:", max_diff)

[0 0 0 0 0 1]         13239.727881         13239.727881             0.000000
[0 0 0 0 0 2]          4627.962169          4627.962169             0.000000
[0 0 0 0 0 3]             0.000000          1439.478742          1439.478742
[0 0 0 0 0 4]             0.000000          1006.758455          1006.758455
[0 0 0 0 1 0]             0.000000             0.000000             0.000000
[0 0 0 0 2 0]         31402.651686         31402.651686             0.000000
[0 0 0 0 2 1]         -7131.621717         -7131.621717             0.000000
[0 0 0 0 2 2]         -2627.521028         -2627.521028             0.000000
[0 0 0 0 3 0]             0.000000         -1462.998223          1462.998223
[0 0 0 0 3 1]            -0.000000         -9825.358434          9825.358434
[0 0 0 0 4 0]             0.000000           463.426455           463.426455
[0 0 0 0 4 1]             0.000000         -6750.610577          6750.610577
[0 0 0 0 5 0]             0.000000          -752.185825           752.185825

### Expand PES using internal coordinates:
- $r_1=r_{CO}-r_{CO}^{eq}(\pi)$
- $r_2=r_{CH_1}-r_{CH_1}^{eq}(\pi)$
- $r_3=r_{CH_2}-r_{CH_2}^{eq}(\pi)$
- $\alpha_1=\alpha_{\angle OCH_1}-\alpha_{\angle OCH_1}^{eq}(\pi)$
- $\alpha_2=\alpha_{\angle OCH_2}-\alpha_{\angle OCH_2}^{eq}(\pi)$
- $\tau-\pi$

Generate expansion for PES

In [4]:
ncoo = 6

max_pow = 6
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("max expansion power:", max_pow)
print("number of expansion terms:", len(deriv_ind))

tau0 = np.pi  # equilibrium value of the `book` dihedral angle

q0 = h2co_AYTY.mep(tau0)

print("equilibrium values of r-alpha-tau coordinates:", q0)

poten_coefs = deriv_list(h2co_AYTY.poten, deriv_ind, q0, if_taylor=True)

max expansion power: 6
number of expansion terms: 924
equilibrium values of r-alpha-tau coordinates: [1.2        1.1        1.1        2.13       2.13       3.14159265]
Time for d= 0 : 7.16 s
Time for d= 1 : 7.33 s
Time for d= 2 : 14.17 s
Time for d= 3 : 24.4 s
Time for d= 4 : 37.9 s
Time for d= 5 : 56.97 s
Time for d= 6 : 81.29 s


Print coefficients

In [11]:
test_file = "_h2co_poten_coefs_6.npy"
test = np.load(test_file)

thresh = 1e-6
for i in range(len(deriv_ind)):
    if np.abs(poten_coefs[i] > thresh):
        print(deriv_ind[i], "%20.6f" % poten_coefs[i])


[0 0 0 0 0 2]          6619.863941
[0 0 0 0 0 4]           876.263322
[0 0 0 0 1 2]          4277.619516
[0 0 0 0 1 5] 632881748775189454468964941824.000000
[0 0 0 0 2 0]         31402.651686
[0 0 0 0 2 3] 75656075492720064.000000
[0 0 0 0 3 3] 13840477035605258046430404149248.000000
[0 0 0 0 4 0]           646.301233
[0 0 0 0 4 1] 54488591485066568.000000
[0 0 0 0 5 0] 10795021518406880.000000
[0 0 0 0 5 1] 5007989271269424243803596783616.000000
[0 0 0 0 6 0] 1172271771020320415992969691136.000000
[0 0 0 1 0 2]          4277.619516
[0 0 0 1 0 5] 632881748775190861843848495104.000000
[0 0 0 1 1 0]         21264.254862
[0 0 0 1 1 2]           214.014360
[0 0 0 1 2 0]         15666.762364
[0 0 0 1 2 1]           185.160713
[0 0 0 1 2 2] 138962621770355664.000000
[0 0 0 1 2 3] 5894901396987752963877195743232.000000
[0 0 0 1 3 0]          2634.993835
[0 0 0 1 4 0] 36363356787168992.000000
[0 0 0 1 4 1] 16251266143872374943642902593536.000000
[0 0 0 2 0 0]         31402.651686
[0 0 0 2 0 3]